In [1]:
#### 패키지 임포트
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
import os
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
pd.set_option('display.max_columns', None)

In [24]:
topwords50 = ['지역', '산림', '문화', '숲', '사람', '교육', '자연', '마을', '이용', '산', '나무', '관광', '생활', '역사', '마음', '활용', '이야기', '미래', '가을', '예술', '청소년', '전통', '삶', '음식', '여름', '소나무', '봄', '이해', '여성', '겨울', '지식', '공감', '정신', '공동체', '사상', '성인', '공예', '정서', '창조', '고급', '남성', '신앙', '교양', '유람', '유람기', '향유', '의식주', '포용', '금송', '성황당', '서낭당', '친화력']
# '학습'을 제거하고 '교육'으로 통합시킴

In [25]:
data = pd.read_csv('나무문화_본문to형태소.csv')
print(data.shape, data.columns)
docs = data['morphs'].to_list()
print(len(docs))

(220262, 1) Index(['morphs'], dtype='object')
220262


In [26]:
#### 각각의 본문이 str타입('나는 어제 밥을 먹었다')일 경우 list(['나', '는', '어제', '밥', '을' '먹다', '이다'])로 바꿔줌
docs2 = docs.copy()
for i in tqdm(range(len(docs))):
    filtered_lst = []
    for w in docs2[i].split():
        if w in topwords50:
            filtered_lst.append(w)
        elif w == '학습':
            filtered_lst.append('교육')
    else:
        docs2[i] = " ".join(filtered_lst)

  0%|          | 0/220262 [00:00<?, ?it/s]

In [ ]:
# #### 중복 제거 잘 됐나 개수로 확인
# print(len(docs[0]))
# print(len(docs2[0]))
# print(docs2[0][0:10])

In [27]:
# TF-IDF 생성
""" CountVectorizer로 TF-IDF를 만들어서 변환하는 방법도 있지만 """
""" 처음부터 Tfidf를 구해주는 함수(TfidfVectorizer)가 있기 때문에 이를 사용할 것임. """

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# 데이터 프레임에서 분석할 문서를 가져와서 쓸 경우 꼭 리스트로 변환해서 담아줘야 함.
corpus = docs2.copy()

In [28]:
# stop_words에는 위에서 조사만 추출해서 만든 불용어 리스트 사용
tfidf = TfidfVectorizer(token_pattern=r'\w{1,}')
# tfidf_v = TfidfVectorizer(stop_words = hyeong_josa2).fit(corpus)
tdm = tfidf.fit_transform(corpus)
print(tdm.shape)

(220262, 50)


In [29]:
# 만든 TF-IDF를 데이터 프레임에 담기 위해 우선 numpy.array()로 변환함
tfidf_array = tdm.toarray()
tfidf_array

array([[0.19473965, 0.81625192, 0.        , ..., 0.        , 0.        ,
        0.20527446],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.83365904, 0.34942847, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [30]:
# 변환한 array를 다시 pd.Dataframe으로 바꿈
tfidf_DF = pd.DataFrame(tfidf_array)
tfidf_DF

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,0.194740,0.816252,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.083833,0.0,0.000000,0.000000,0.085143,0.000000,0.191805,0.313162,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.191372,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.152456,0.00000,0.000000,0.000000,0.0,0.21135,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.205274
1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.170598,0.0,0.000000,0.276607,0.173264,0.000000,0.000000,0.212424,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.905119,0.000000,0.00000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.232315,0.0,0.629663,0.376675,0.000000,0.000000,0.000000,0.289273,0.0,0.000000,0.0,0.168298,0.000000,0.0,0.0,0.0,0.343658,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.421386,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.000000,0.000000,0.000000,0.0,0.000000,0.565129,0.000000,0.0,0.000000,0.0,0.078008,0.0,0.000000,0.126482,0.158454,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.329430,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.709312,0.00000,0.000000,0.000000,0.0,0.00000,0.0,0.147618,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.000000,0.522019,0.365836,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.108904,0.000000,0.000000,0.133518,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.750981,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220257,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.233974,0.0,0.000000,0.0,0.000000,0.000000,0.105950,0.832925,0.000000,0.324741,0.0,0.000000,0.0,0.000000,0.220273,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.094856,0.24694,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.127719
220258,0.000000,0.000000,0.000000,0.0,0.145279,0.000000,0.000000,0.0,0.000000,0.0,0.083863,0.0,0.255712,0.000000,0.042587,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.708310,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.00000,0.0,0.634788,0.0,0.0,0.0,0.0,0.0,0.000000
220259,0.833659,0.349428,0.000000,0.0,0.000000,0.000000,0.182353,0.0,0.000000,0.0,0.071776,0.0,0.291812,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.159265,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.130530,0.00000,0.130192,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
220260,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.405020,0.0,0.000000,0.000000,0.102837,0.000000,0.463330,0.504321,0.0,0.193024,0.0,0.000000,0.213802,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.188845,0.221571,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.367324,0.231769,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000


In [32]:
# 이 데이터프레임의 컬럼명은 숫자로 되어 있음. 이를 feature_names로 바꿔줘야 함.
featurenames = tfidf.get_feature_names_out()
tfidf_DF.columns = featurenames

In [33]:
tfidf_DF

,가을,겨울,고급,공감,공동체,공예,관광,교양,교육,금송,나무,남성,마을,마음,문화,미래,봄,사람,사상,산,산림,삶,생활,서낭당,성인,소나무,숲,신앙,여름,여성,역사,예술,유람,음식,의식주,이야기,이용,이해,자연,전통,정서,정신,지식,지역,창조,청소년,친화력,포용,향유,활용
0,0.194740,0.816252,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.083833,0.0,0.000000,0.000000,0.085143,0.000000,0.191805,0.313162,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.191372,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.152456,0.00000,0.000000,0.000000,0.0,0.21135,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.205274
1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.170598,0.0,0.000000,0.276607,0.173264,0.000000,0.000000,0.212424,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.905119,0.000000,0.00000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.232315,0.0,0.629663,0.376675,0.000000,0.000000,0.000000,0.289273,0.0,0.000000,0.0,0.168298,0.000000,0.0,0.0,0.0,0.343658,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.421386,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.000000,0.000000,0.000000,0.0,0.000000,0.565129,0.000000,0.0,0.000000,0.0,0.078008,0.0,0.000000,0.126482,0.158454,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.329430,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.709312,0.00000,0.000000,0.000000,0.0,0.00000,0.0,0.147618,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.000000,0.522019,0.365836,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.108904,0.000000,0.000000,0.133518,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.750981,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220257,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.233974,0.0,0.000000,0.0,0.000000,0.000000,0.105950,0.832925,0.000000,0.324741,0.0,0.000000,0.0,0.000000,0.220273,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.094856,0.24694,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.127719
220258,0.000000,0.000000,0.000000,0.0,0.145279,0.000000,0.000000,0.0,0.000000,0.0,0.083863,0.0,0.255712,0.000000,0.042587,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.708310,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.00000,0.0,0.634788,0.0,0.0,0.0,0.0,0.0,0.000000
220259,0.833659,0.349428,0.000000,0.0,0.000000,0.000000,0.182353,0.0,0.000000,0.0,0.071776,0.0,0.291812,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.159265,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.130530,0.00000,0.130192,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
220260,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.405020,0.0,0.000000,0.000000,0.102837,0.000000,0.463330,0.504321,0.0,0.193024,0.0,0.000000,0.213802,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.188845,0.221571,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.367324,0.231769,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000


In [34]:
tfidf_DF[tfidf_DF['나무']==1]

,가을,겨울,고급,공감,공동체,공예,관광,교양,교육,금송,나무,남성,마을,마음,문화,미래,봄,사람,사상,산,산림,삶,생활,서낭당,성인,소나무,숲,신앙,여름,여성,역사,예술,유람,음식,의식주,이야기,이용,이해,자연,전통,정서,정신,지식,지역,창조,청소년,친화력,포용,향유,활용
181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
358,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
868,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
219317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
219319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
219371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# 각 단어에 대한 TF-IDF의 합
word_TFIDF_sum = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    'tf-idf': tdm.sum(axis=0).flat
})
word_TFIDF_sum = word_TFIDF_sum.sort_values(by='tf-idf', ascending=False).reset_index(drop=True)
word_TFIDF_sum

In [ ]:
# 각 단어에 대한 TF-IDF의 합
word_TFIDF_average = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    'tf-idf': tdm.mean(axis=0).flat
})
word_TFIDF_average = word_TFIDF_average.sort_values(by='tf-idf', ascending=False).reset_index(drop=True)
word_TFIDF_average

In [43]:
word_TFIDF_average.to_excel("나무문화_tfidf_top50.xlsx", index=False)